In [1]:
url = "https://raw.githubusercontent.com/chiarorosa/ia_aprendizado_maquina_basico/main/ml-dataset/kaggle-basico/attrition_Rate_Company.csv"

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # Avaliação de Acurácia
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


In [2]:
df = pd.read_csv(url)
df.head()

df.dropna(inplace=True)

labelEnconder = LabelEncoder()
categorizacaoColunas = ['BusinessTravel', 'EducationField', 'Department', 'Gender', 'JobRole', 'MaritalStatus','Attrition','Over18']

for coluna in categorizacaoColunas:
    df[coluna] = labelEnconder.fit_transform(df[coluna])
    
X = df.drop('Attrition', axis=1)
y = df['Attrition']







In [3]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)


# Dividir em dois conjuntos, treinamento e teste, usando pareto 80/20
X_train, X_test, y_train, y_test = train_test_split(
    x_scaled,
    y,
    test_size=0.2, # pareto
    random_state=42 # reproduzivel
)

In [15]:
grade = {
    'n_neighbors': range(1, 50),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p':[1,2],
    'leaf_size': range(1, 50),
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']
}
knm = RandomizedSearchCV(
    KNeighborsClassifier(),
    grade,
    cv=15,
    scoring='accuracy',
)

knm.fit(X_train, y_train)

melhor = knm.best_estimator_

y_pred = melhor.predict(X_test)

acuracia = accuracy_score(y_test, y_pred)

print(f'Acurácia: {acuracia * 100:.2f}')
print(f'Melhor modelo: {melhor}')
print(f'Parâmetros do melhor modelo: {knm.best_params_}')

guardaAcc = 0
with open('Melhor Acurracia KNeighbors.txt', 'r+', encoding='utf-8') as f:
        try:
            linha = f.readline().strip()
            if linha.startswith('Acurácia:'):
                guardaAcc = float(linha.split(': ')[1]) / 100
                print(f'Acurácia anterior: {guardaAcc * 100:.2f}')
                if acuracia > guardaAcc:
                    f.seek(0) 
                    f.write(f'Acurácia: {acuracia * 100:.2f} \n')
                    
                    f.truncate() 
                    print(f'Acurácia atualizada: {acuracia * 100:.2f}')
                else:
                    print('Acurácia não atualizada')
        except Exception as e:
            print(f"Erro ao ler o arquivo: {e}")
       


Acurácia: 97.44
Melhor modelo: KNeighborsClassifier(algorithm='brute', leaf_size=37, metric='euclidean',
                     n_neighbors=3, p=1, weights='distance')
Parâmetros do melhor modelo: {'weights': 'distance', 'p': 1, 'n_neighbors': 3, 'metric': 'euclidean', 'leaf_size': 37, 'algorithm': 'brute'}
Acurácia anterior: 99.30
Acurácia não atualizada
